In [ ]:
pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fm = keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels) = fm.load_data() 

In [ ]:
# Scaling the dataset
train_images = train_images/255.0
test_images = test_images/255.0


In [ ]:
test_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
train_images[0].shape

(28, 28, 1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([ 
                            keras.layers.Conv2D(
                                filter = hp.int('conv_1_filter',min_value = 32, max_value = 128, step=16),
                                kernel_size = hp.choice('conv_1_kernal',values = [3,5]),
                                activation = 'relu',
                                input_shape = (28,28,1)
                                ),
                            
                            
  ])

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory = 'output1', project_name = 'Mnist Fashion')

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3966 - accuracy: 0.8570 - val_loss: 0.2959 - val_accuracy: 0.8893
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2561 - accuracy: 0.9051 - val_loss: 0.2560 - val_accuracy: 0.9042
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2002 - accuracy: 0.9262 - val_loss: 0.2505 - val_accuracy: 0.9093


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.6876 - accuracy: 0.7784 - val_loss: 0.4401 - val_accuracy: 0.8360
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4410 - accuracy: 0.8369 - val_loss: 0.4235 - val_accuracy: 0.8452
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4168 - accuracy: 0.8463 - val_loss: 0.4553 - val_accuracy: 0.8298


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5925 - accuracy: 0.7896 - val_loss: 0.4838 - val_accuracy: 0.8248
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4511 - accuracy: 0.8342 - val_loss: 0.4460 - val_accuracy: 0.8405
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4237 - accuracy: 0.8463 - val_loss: 0.4390 - val_accuracy: 0.8465


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3861 - accuracy: 0.8604 - val_loss: 0.2985 - val_accuracy: 0.8887
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2485 - accuracy: 0.9090 - val_loss: 0.2557 - val_accuracy: 0.9052
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1868 - accuracy: 0.9299 - val_loss: 0.2499 - val_accuracy: 0.9105


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4212 - accuracy: 0.8462 - val_loss: 0.3164 - val_accuracy: 0.8898
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2751 - accuracy: 0.9000 - val_loss: 0.2727 - val_accuracy: 0.8987
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2209 - accuracy: 0.9179 - val_loss: 0.2536 - val_accuracy: 0.9068


INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3965056   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 4,004,938
Trainable params: 4,004,938
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1371 - accuracy: 0.9490 - val_loss: 0.2617 - val_accuracy: 0.9123
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0974 - accuracy: 0.9634 - val_loss: 0.3015 - val_accuracy: 0.9070
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0692 - accuracy: 0.9749 - val_loss: 0.3302 - val_accuracy: 0.9100
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0514 - accuracy: 0.9816 - val_loss: 0.3735 - val_accuracy: 0.9118
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0397 - accuracy: 0.9858 - val_loss: 0.4241 - val_accuracy: 0.9153
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0313 - accuracy: 0.9894 - val_loss: 0.4921 - val_accuracy: 0.9083
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0284 - accuracy: 0.9907 - val_loss: 0.5127 - val_accuracy